# Depth LFP Analysis Pipeline 

Welcome to the script for generating parameter dictionaries for the recording sessions in your experiment folder. Please follow the upcoming steps in this notebook for further instructions. 

## 1) Import the packages required for running the script

Please run the block of code to import the Python packages that are required for running the rest of this script. 

In [1]:
#Import required packages
import os, ipywidgets, pickle, h5py, shutil
from utils.load_intan_rhd_format import * 
from utils.reading_utils import *
from ipywidgets import Layout, HBox, VBox
from IPython.display import display
from utils.experiment_classes import *
from utils.filtering import * 
from spikeSortingUtils.klusta_pre_and_post_processing_utils import * 
from utils.notebook_utils import * 
from copy import deepcopy

## 2) Enter general parameters for the experiment.

In [2]:
#Creating widgets for the user input on the general parameters for the experiment

##Main path for the data 

mp_html = ipywidgets.HTML(value = "<p><b>Path to the data of the experiment:</b><br />Enter the path to the folder (with no '/' at the end) that is hierarchically right above the folders of the recording sessions</p>")
mp = ipywidgets.Text(value = "folder", placeholder = "Enter path for data", disabled = False)
display(VBox([mp_html, mp]))

##Type of the experiment

#et_html = ipywidgets.HTML(value = "<b>Type of the experiment</b>")
#et = ipywidgets.Dropdown(options=['Acute', 'Chronic'], 
#                   value = 'Chronic',  disabled = False)
#display(VBox([et_html, et]))


##File format
ff_html = ipywidgets.HTML(value = "<p><b>File format:</b><br />(dat for .dat, cont for .continuous, rhd for .rhd)</p>")
ff = ipywidgets.Text(value = 'dat', placeholder = 'Enter file format',
             disabled = False)
display(VBox([ff_html,ff]))

##Amplifier port
ap_html = ipywidgets.HTML(value = "<b>The port to which the amplifier is connected</b>")
ap = ipywidgets.Dropdown(options=['A', 'B', 'C', 'D'], 
                   value = 'A',  disabled = False)
display(VBox([ap_html, ap]))

##Whisker Stim Path
wsp = ipywidgets.IntText(value = 0, disabled = False)
display(VBox([ipywidgets.HTML(value = "<b>Enter the index of the digital input channel where the whisker stimulation trigger is kept</b>"), wsp]))

##Probe file path 
pi_html = ipywidgets.HTML(value = "<b>Probe name:</b>")
pi = ipywidgets.Text(value = '64_channel_single_shank_v2', placeholder = 'probe_file',
             disabled = False)
display(VBox([pi_html,pi]))

## 3) Enter parameters related to evoked LFP analysis

In [3]:
#Creating widgets for the user input on the parameters related to the whisker stim evoked LFP analysis

##Downsampled frequency
ds = ipywidgets.IntText(value = 1000, disabled = False)
display(VBox([ipywidgets.HTML(value = "<b>Enter the desired frequency of downsampled data for LFP analysis. </b>"), ds]))

##whiskerEvokedPre

wpre = ipywidgets.FloatText(value = 0.025, disabled = False)
display(VBox([ipywidgets.HTML(value = "<b>Enter time taken prior to the whisker stimulus trigger (in s)</b>"), wpre]))

##whiskerEvokedPost

wpost = ipywidgets.FloatText(value = 0.100, disabled = False)
display(VBox([ipywidgets.HTML(value = "<b>Enter time taken post the whisker stimulus trigger (in s)</b>"), wpost]))

#low_pass_freq
lp = ipywidgets.FloatText(value = 300, disabled = False)
display(VBox([ipywidgets.HTML(value = "<b> Enter the cutoff frequency of the low pass filter to extract LFP from data (in Hz)"), lp]))

#Low pass filter order
lp_order = ipywidgets.IntText(value = 4, disabled = False)
display(VBox([ipywidgets.HTML(value = "<b> Enter the order of the Butterworth low pass filter used for the evoked LFP analysis (in Hz)"), lp_order]))

##cutBeginning
cb = ipywidgets.FloatText(value = 1, disabled = False)
display(VBox([ipywidgets.HTML(value = "<b>Enter the time to be cut from the beginning of the recording (in s)</b>"), cb]))

##cutEnd
ce = ipywidgets.FloatText(value = 1, disabled = False)
display(VBox([ipywidgets.HTML(value = "<b>Enter the time to be cut from the end of the recording (in s )"), ce]))



# 4) Enter parameters related to spike sorting

If you are intending to do spike-sorting on this data, please set the spike-sorting parameters. Otherwise, set the boolean parameter "do_spike_sorting" to False below. 

In [4]:
#Creating widgets for the user input on the parameters related to spike sorting

##samplesBefore
sb = ipywidgets.FloatText(value = 0.5, disabled = False)
display(VBox([ipywidgets.HTML(value = '<b>Enter the length of waveform to be taken before the threshold crossing (in ms)'), sb]))

##samplesAfter
sa = ipywidgets.FloatText(value = 1.5, disabled = False)
display(VBox([ipywidgets.HTML(value = "<b>Enter the length of waveform to be taken after the threshold crossing (in ms)"), sa]))

##lowCutoff
lc = ipywidgets.FloatText(value = 500., disabled = False)
display(VBox([ipywidgets.HTML(value = "<b>Enter the lower cutoff frequency for the bandpass filter to be applied on the raw data (in Hz)"), lc]))

##thresholdingCoefficient
tc = ipywidgets.FloatText(value = 4.5, disabled = False)
display(VBox([ipywidgets.HTML(value = "<b>Enter the thresholding coefficient (in terms of multiple of baseline noise rms) to be used for spike detection"), tc]))

##Bandpass filter order 
fo = ipywidgets.IntText(value = 4)
display(VBox([ipywidgets.HTML(value = "<b>Enter the order of the Butterworth bandpass filter used on the raw data"), fo]))


# 5) Generate the parameters dictionaries

Please run the block of the code in order to generate the parameters dictionary for each recording session (paramsDict.p) based on the input that you have provided above. 

In [5]:
experiment = Experiment(mp.value)
experiment.add_sessions_in_dir()

for session_index in experiment.sessions:
    session = experiment.sessions[session_index]
    session.amplifier_port = ap.value
    session.setTrigChannels(wsp.value)
    
    header_dir = session.dir + '/info.rhd'
    header = read_data(header_dir)
    session.sample_rate = header['frequency_parameters']['amplifier_sample_rate']
    session.spike_samples_before = int(sb.value * session.sample_rate / 1000)
    session.spike_samples_after = int(sa.value * session.sample_rate / 1000)
    session.downsampling_factor = int(session.sample_rate / ds.value)
    session.get_duration()

experiment.amplifier = session.amplifier
experiment.createProbe(mp.value, pi.value)   

#General parameters   
experiment.fileformat = ff.value

#Parameters related to spike sorting 
experiment.low_cutoff_bandpass = lc.value
experiment.threshold_coeff = tc.value 
experiment.bandfilter_order = fo.value

#Parameters related to evoked LFP analysis 
experiment.cut_beginning = cb.value 
experiment.cut_end = ce.value
experiment.low_pass_freq = lp.value
experiment.low_pass_order = lp_order.value 
experiment.whisker_evoked_pre = wpre.value
experiment.whisker_evoked_post = wpost.value

mBY05_awake_190909_165527
Do spike detection, sorting and post-processing for this session? (y/n)y
Do whisker stimulation evoked analysis for this session? (y/n)n
Which channels will be used for software referencing to detect spikes?0,1,2,3,5,6,7,8,11,12,13,14,15,16,17,18,19,20,21,22,23,24
Which channels are dead?25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63

Reading Intan Technologies RHD2000 Data File, Version 1.5

Found 64 amplifier channels.
Found 3 auxiliary input channels.
Found 1 supply voltage channel.
Found 0 board ADC channels.
Found 0 board digital input channels.
Found 0 board digital output channels.
Found 0 temperature sensors channels.

Header file contains no data.  Amplifiers were sampled at 30.00 kS/s.
Done!  Elapsed time: 0.0 seconds


# 6) Read and analyze stimulus evoked LFPs, generate dictionaries for spike sorting analysis

cwd = os.getcwd()
os.chdir(mp.value)
experiment = pickle.load(open('experiment_params.p', 'rb'))
os.chdir(cwd)


In [6]:
f = h5py.File(experiment.dir + '/preprocessing_results.hdf5', 'a')
preprocessing_files_dir = mp.value + '/preprocessing_files'

#Create the folder for the preprocessing results
if not os.path.exists(preprocessing_files_dir):
    os.mkdir(preprocessing_files_dir)

for session_index in experiment.sessions:
    session = experiment.sessions[session_index]
    print("session:" + session.name)
    
    #Create the group for session in the hdf file
    if session.name not in f:
        ses_grp = f.create_group(session.name)
        ds_lfp_grp = ses_grp.create_group("downsampled_lfp") #Folder for the downsampled LFPs

    #Create the folder for session in the preprocessing results folder
    session_preprocessing_files_dir = preprocessing_files_dir + '/' + session.name
    if not os.path.exists(session_preprocessing_files_dir):
        os.mkdir(session_preprocessing_files_dir)
    
    #Create the group for channel groups in the hdf file
    for group in range(experiment.probe.nr_of_groups):
        ch_grp = ses_grp.create_group('group_{:g}'.format(group))
        nr_of_electrodes = len(experiment.probe.id[group])
        
        probe_id = deepcopy(experiment.probe.id)
        channels = probe_id[group]
        if session.dead_channels != ['']:
            for dead_channel in sorted(session.dead_channels, reverse=True):
                channels.remove(dead_channel)
        session.good_channels = channels
        
        for trode in channels:
            electrode_data = read_channel(session, group, trode, [0,-1])
            electrode_data = signal.decimate(electrode_data, session.downsampling_factor)
            ds_lfp_grp.create_dataset("channel_{:g}_downsampled_LFP".format(trode), data = electrode_data)    
    
    #Preprocessing evoked LFPs, if applicable
    if session.preferences['do_whisker_stim_evoked'] == 'y':
        print("Analyzing evoked LFPs")
        evoked_LFP_preprocessing_files_dir = session_preprocessing_files_dir + '/evoked_LFP_analysis'
        if not os.path.exists(evoked_LFP_preprocessing_files_dir):
            os.mkdir(evoked_LFP_preprocessing_files_dir)

        read_stimulus_trigger(session)

        for group in range(experiment.probe.nr_of_groups):
            print("Channel group: " + str(group))
            read_evoked_lfp(group,session)
    
    #Preparing files for spike sorting, if applicable
    if session.preferences['do_spike_analysis'] == 'y':
        print("Pre-processing data for spike sorting")
        spike_sorting_preprocessing_files_dir = session_preprocessing_files_dir + '/spike_sorting'

        if not os.path.exists(spike_sorting_preprocessing_files_dir):
            os.mkdir(spike_sorting_preprocessing_files_dir)

        for group in range(experiment.probe.nr_of_groups):
            if not os.path.exists(spike_sorting_preprocessing_files_dir + '/group_{:}'.format(group)):
                os.mkdir(spike_sorting_preprocessing_files_dir + '/group_{:}'.format(group))

            print("Channel group: " + str(group))
            create_linear_prb_file(group, session)
            create_prm_file(group, session)
            read_group_into_dat_file(session, group, spike_sorting_preprocessing_files_dir)

f.close()

session:mBY05_awake_190909_165527


/home/baran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


Pre-processing data for spike sorting
Channel group: 0


/home/baran/Desktop/github/depth_recordings/spikeSortingUtils/klusta_pre_and_post_processing_utils.py:47: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if session.dead_channels != ['']:
/home/baran/Desktop/github/depth_recordings/utils/reading_utils.py:156: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if (session.ref_channels != ['']):


In [ ]:
pickle.dump(experiment, open((mp.value + '/experiment_params.p'), 'wb'))

# 7) Add new days for the chronic recording experiments (Start from here if files for this experiment had already been generated before)

In [ ]:
main_folder = '/home/baran/Dropbox (Yanik Lab)/Layer 1 Project/Electrophysiology/Experiments/mBY05'
sys.path.insert(0, main_folder)
experiment_file = open((main_folder + '/experiment_params.p'), 'rb')
experiment = pickle.load(experiment_file)
f = h5py.File(experiment.dir + '/preprocessing_results.hdf5', 'r+')
preprocessing_files_dir = experiment.dir + '/preprocessing_files'
probe_name = str(experiment.probe.__class__).split('.')[1][:-2]

In [ ]:
#Obtaining the sessions that are already saved in the experiment dictionary
existing_sessions = []
for i in range(len(experiment.sessions)):
    existing_sessions.append(experiment.sessions[i].name)

In [ ]:
existing_sessions

In [ ]:
#Adding the new days of recording since the last analysis
sessions = glob(main_folder + '/*[!preprocessing]')
for session in sessions:
    if session not in existing_sessions:
        experiment.add_session(day)
        
for session_index in experiment.sessions:
    session = experiment.sessions[session_index]
    if session.name not in existing_sessions:
        print("session:" + session.name)
        ses_grp = f.create_group(session.name)
        experiment.add_sessions_in_dir()
        
        subExperiment_preprocessing_files_dir = preprocessing_files_dir + '/' + subExperiment.name
        if not os.path.exists(subExperiment_preprocessing_files_dir):
            os.mkdir(subExperiment_preprocessing_files_dir)
        
        for session_index in subExperiment.sessions:
            session = subExperiment.sessions[session_index]
            print("Session:" + session.name)
            ses_grp = subExperiment_grp.create_group(session.name)
        
            session_preprocessing_files_dir = subExperiment_preprocessing_files_dir + '/' + session.name
            if not os.path.exists(session_preprocessing_files_dir):
                os.mkdir(session_preprocessing_files_dir)
        
            for group in range(experiment.probe.nr_of_groups):
                ch_grp = ses_grp.create_group('group_{:g}'.format(group))
        
            if session.preferences['do_whisker_stim_evoked'] == 'y':
                print("Analyzing evoked LFPs")
                evoked_LFP_preprocessing_files_dir = session_preprocessing_files_dir + '/evoked_LFP_analysis'
                if not os.path.exists(evoked_LFP_preprocessing_files_dir):
                    os.mkdir(evoked_LFP_preprocessing_files_dir)
            
                stim_timestamps = read_stimulus_trigger(session)
                for group in range(experiment.probe.nr_of_groups):
                    print("Channel group: " + str(group))
                    read_evoked_lfp(group,session)
        
            if session.subExperiment.preferences['do_spike_analysis'] == 'y':
                print("Pre-processing data for spike sorting")
                spike_sorting_preprocessing_files_dir = session_preprocessing_files_dir + '/spike_sorting'
                if not os.path.exists(spike_sorting_preprocessing_files_dir):
                    os.mkdir(spike_sorting_preprocessing_files_dir)
            
                for group in range(experiment.probe.nr_of_groups):
                    if not os.path.exists(spike_sorting_preprocessing_files_dir + '/group_{:}'.format(group)):
                        os.mkdir(spike_sorting_preprocessing_files_dir + '/group_{:}'.format(group))
                    print("Channel group: " + str(group))
                    create_prm_file(group, session)
                    create_linear_prb_file(group, session)
                    read_group_into_dat_file(session, group, spike_sorting_preprocessing_files_dir)
    else:
        pass 

f.close()

In [ ]:
pickle.dump(experiment, open((main_folder + '/experiment_params.p'), 'wb'))

Notebook written by Baran Yasar in 04/2017. Please contact him in person or e-mail at yasar@biomed.ee.ethz.ch in case of any questions. 

---Updated in 07/2018 by Baran Yasar